In [3]:
import boto3
import xmltodict
import json
import datetime
import sys
import os
import pandas as pd
import pprint
import time

In [4]:
pp = pprint.PrettyPrinter(indent=4)

In [5]:
AWS_SECRET = os.environ['AWS_SECRET']
AWS_REGION = os.environ['AWS_REGION']
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY']
DATA_COLLECTION_LOCATION=os.environ['DATA_COLLECTION_LOCATION']

In [6]:
EXPERIMENT_NAME="SimpleExaCT_PoC"

## Authenticate

In [7]:
create_hits_in_live = False
environments = {
        "live": {
            "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
            "preview": "https://www.mturk.com/mturk/preview",
            "manage": "https://requester.mturk.com/mturk/manageHITs",
            "reward": "0.00"
        },
        "sandbox": {
            "endpoint": "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
            "preview": "https://workersandbox.mturk.com/mturk/preview",
            "manage": "https://requestersandbox.mturk.com/mturk/manageHITs",
            "reward": "0.11"
        },
}
mturk_environment = environments["live"] if create_hits_in_live else environments["sandbox"]

In [8]:
session = boto3 #.Session(profile_name='mturk')
mturk = session.client(
    service_name='mturk',
    region_name=AWS_REGION,
    aws_access_key_id = AWS_ACCESS_KEY, 
    aws_secret_access_key = AWS_SECRET,
    endpoint_url=mturk_environment['endpoint'],
)

In [9]:
print("I have $" + mturk.get_account_balance()['AvailableBalance'] + " in my account")

I have $10000.00 in my account


### Create HITs

In [10]:
question_template = open("templates/question_template.xml", "r").read()

In [11]:
html_layout = open('./templates/HIT2.html', 'r').read()

In [12]:
# Example of using qualification to restrict responses to Workers who have had
# at least 80% of their assignments approved. See:
# http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
participant_requirements = {
    'QualificationTypeId': '000000000000000000L0',
    'Comparator': 'GreaterThanOrEqualTo',
    'IntegerValues': [80],
    'RequiredToPreview': True,
}

In [13]:
def create_hit(URL,challenge_response,qualifications):
    html_with_link=html_layout.replace('EXPERIMENTLINK',URL)
    question_assembled=question_template.format(html_with_link)
    # Create the HIT
    response = mturk.create_hit(
        MaxAssignments=1,
        # How long the task will be available on MTurk 
        LifetimeInSeconds=600,
        # How long Participants have to complete each item
        AssignmentDurationInSeconds=600,
        Reward=mturk_environment['reward'],
        Title='This is a desktop Virtual Reality experiment',
        Keywords='VR, slow, research',
        Description='In this experiment you will slowly move through a building and will have to find specific targets.',
        Question=question_assembled,
        QualificationRequirements=qualifications,
    )
    # The response included several fields that will be helpful later
    hit_type_id = response['HIT']['HITTypeId']
    hit_id = response['HIT']['HITId']
    print("\nCreated HIT: {} with response {}".format(hit_id,challenge_response))
    print("\nYou can work the HIT here:")
    hit_url=mturk_environment['preview'] + "?groupId={}".format(hit_type_id)
    print(hit_url)
    return hit_id, hit_url

In [14]:
df=pd.read_csv("../../experiment/data/AllCodesNew.csv")

In [15]:
trial_set=df.iloc[0:3]

In [16]:
trial_set

,Unnamed: 0,Phase,Condition,accessCode,exitCode,Reward,ExpID,Bonus
0,0,E1,1,[ExpID=78b9dc93aa6a4cf074d05d28ed1f1373&group=1],51c53943,undefined,78b9dc93aa6a4cf074d05d28ed1f1373,NaN
1,1,E1,1,[ExpID=40186ebc8c2b0ee62a7b7a9e23fae41d&group=1],8489889e,undefined,40186ebc8c2b0ee62a7b7a9e23fae41d,NaN
2,2,E1,1,[ExpID=5b764a9f6f1f8659f0c025c9c3aa6a06&group=1],120aa203,undefined,5b764a9f6f1f8659f0c025c9c3aa6a06,NaN


In [17]:
response = mturk.create_qualification_type(
    Name=f'Participated in the experiment {EXPERIMENT_NAME}',
    Keywords='participation',
    Description='Qualification used to keep track of participation',
    QualificationTypeStatus='Active',
    RetryDelayInSeconds=600,
    AutoGranted=True,
    AutoGrantedValue=1
)
try:
    print (json.dumps(response, indent=2))
except:
    print(response)

{'QualificationType': {'QualificationTypeId': '3BR5SIVR7VUQOEDI0JFGCDU88RBEGQ', 'CreationTime': datetime.datetime(2021, 3, 14, 12, 55, 22, tzinfo=tzlocal()), 'Name': 'Participated in the experiment SimpleExaCT_PoC', 'Description': 'Qualification used to keep track of participation', 'Keywords': 'participation', 'QualificationTypeStatus': 'Active', 'RetryDelayInSeconds': 600, 'IsRequestable': True, 'AutoGranted': True, 'AutoGrantedValue': 1}, 'ResponseMetadata': {'RequestId': '43c9da09-6de2-4466-9a2f-86bfdb185851', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '43c9da09-6de2-4466-9a2f-86bfdb185851', 'content-type': 'application/x-amz-json-1.1', 'content-length': '378', 'date': 'Sun, 14 Mar 2021 12:55:22 GMT'}, 'RetryAttempts': 0}}


In [18]:
qual_type=response['QualificationType']['QualificationTypeId']

In [19]:
participation_qual={
            'QualificationTypeId': qual_type,
            'Comparator': 'DoesNotExist', #|'LessThan',|'LessThanOrEqualTo'|'GreaterThan'|'GreaterThanOrEqualTo'|'EqualTo'|'NotEqualTo'|'Exists'|'DoesNotExist'|'In'|'NotIn',
            #'IntegerValues': [
            #    1,
            #],
            'ActionsGuarded': 'Accept' #|'PreviewAndAccept'|'DiscoverPreviewAndAccept'
        }

In [20]:
qualifications_required=[participant_requirements,participation_qual]

In [21]:
output = pd.DataFrame()
for idx,trial in trial_set.iterrows():
    URL=f"{DATA_COLLECTION_LOCATION}/index.html?ExpID={trial['ExpID']}&group={trial['Condition']}"
    hit_id, hit_url=create_hit(URL,trial['exitCode'],qualifications_required)
    results={"HITId":hit_id,"URL":URL,"RESP_CHALLENGE":trial['exitCode'], "HIT_URL":hit_url}
    output.append(results, ignore_index=True)


Created HIT: 3LB1BGHFL2W0C0U8T78Z6OGFT12YTZ with response 51c53943

You can work the HIT here:
https://workersandbox.mturk.com/mturk/preview?groupId=3MY6FNYS863U5KF2USGZH62EPF52BA

Created HIT: 3XJOUITW8URR3Y3M2XR5T2DDUSLTQR with response 8489889e

You can work the HIT here:
https://workersandbox.mturk.com/mturk/preview?groupId=3MY6FNYS863U5KF2USGZH62EPF52BA

Created HIT: 32TZXEA1OLKJ594F2H0IKTTD6HD149 with response 120aa203

You can work the HIT here:
https://workersandbox.mturk.com/mturk/preview?groupId=3MY6FNYS863U5KF2USGZH62EPF52BA


In [50]:
for item in results:
    # Get the status of the HIT
    hit = mturk.get_hit(HITId=item['HITId'])
    item['status'] = hit['HIT']['HITStatus']
    # Get a list of the Assignments that have been submitted
    assignmentsList = mturk.list_assignments_for_hit(
        HITId=item['HITId'],
        #AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    responses = []
    #pp.pprint(assignmentsList)
    for assignment in assignments:
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        answer = answer_dict['QuestionFormAnswers']['Answer']['FreeText']
        status=assignment['AssignmentStatus']
        #print(assignment)
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus']=='Submitted':
            print("### Evaluating assignment")
            
            if answer == item["RESP_CHALLENGE"]:
                status='Approved'
                response = mturk.approve_assignment(
                    AssignmentId=assignment_id,
                    RequesterFeedback='##Thank you for participating in this study',
                    OverrideRejection=False
                )
            else:
                status='Rejected'
                response = mturk.reject_assignment(
                    AssignmentId=assignment_id,
                    RequesterFeedback='@@Sorry, could\'t validate your response'
                )
            response = mturk.associate_qualification_with_worker(
                QualificationTypeId=qual_type,
                WorkerId=worker_id,
                IntegerValue=1,
                SendNotification=False
            )
            responses.append({'answer':answer,'WorkerId':worker_id,'AssignmentId':assignment_id,'Status':status})
        else:
            responses.append({'answer':answer,'WorkerId':worker_id,'AssignmentId':assignment_id,'Status':status})
            #pp.pprint(assignment)
    # Add the answers that have been retrieved for this item
    item['responses'] = responses
print(json.dumps(results,indent=2))

EndpointConnectionError: Could not connect to the endpoint URL: "https://mturk-requester-sandbox.us-east-1.amazonaws.com/"

In [47]:
print(json.dumps(results,indent=2))

[
  {
    "HITId": "3LB1BGHFL2W0C0U8T78Z6OGFT12YTZ",
    "URL": "http://zoll-experiment.s3.amazonaws.com/index.html?ExpID=78b9dc93aa6a4cf074d05d28ed1f1373&group=1",
    "RESP_CHALLENGE": "51c53943",
    "HIT_URL": "https://workersandbox.mturk.com/mturk/preview?groupId=3MY6FNYS863U5KF2USGZH62EPF52BA",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "responses": []
  },
  {
    "HITId": "3XJOUITW8URR3Y3M2XR5T2DDUSLTQR",
    "URL": "http://zoll-experiment.s3.amazonaws.com/index.html?ExpID=40186ebc8c2b0ee62a7b7a9e23fae41d&group=1",
    "RESP_CHALLENGE": "8489889e",
    "HIT_URL": "https://workersandbox.mturk.com/mturk/preview?groupId=3MY6FNYS863U5KF2USGZH62EPF52BA",
    "status": "Reviewable",
    "assignments_submitted_count": 1,
    "responses": [
      {
        "answer": "51c53943",
        "WorkerId": "AO18L4K2NAU0S",
        "AssignmentId": "3G2UL9A02DEDI2LO0NF76L0K4DD67N",
        "Status": "Rejected"
      }
    ]
  },
  {
    "HITId": "32TZXEA1OLKJ594F2H0IKTT

# Check HITs

### List assignments

In [48]:
all_hits=mturk.list_hits()

In [49]:
for h in all_hits['HITs']:
    print(h['HITId'])
    print(h['Title'])
    print(h['NumberOfAssignmentsCompleted'])
    print(pp.pprint(h))

3XJOUITW8URR3Y3M2XR5T2DDUSLTQR
This is a desktop Virtual Reality experiment
1
{   'AssignmentDurationInSeconds': 600,
    'AutoApprovalDelayInSeconds': 2592000,
    'CreationTime': datetime.datetime(2021, 3, 14, 12, 55, 27, tzinfo=tzlocal()),
    'Description': 'In this experiment you will slowly move through a building '
                   'and will have to find specific targets.',
    'Expiration': datetime.datetime(2021, 3, 14, 13, 5, 27, tzinfo=tzlocal()),
    'HITGroupId': '3RC5MYEBOEU78M0IKQNQ3GAJG35TP6',
    'HITId': '3XJOUITW8URR3Y3M2XR5T2DDUSLTQR',
    'HITReviewStatus': 'NotReviewed',
    'HITStatus': 'Reviewable',
    'HITTypeId': '3MY6FNYS863U5KF2USGZH62EPF52BA',
    'Keywords': 'VR, slow, research',
    'MaxAssignments': 1,
    'NumberOfAssignmentsAvailable': 0,
    'NumberOfAssignmentsCompleted': 1,
    'NumberOfAssignmentsPending': 0,
    'QualificationRequirements': [   {   'ActionsGuarded': 'PreviewAndAccept',
                                         'Comparator': 'Gre

In [33]:
for h in all_hits['HITs']:
    print(h['HITId'])
    print(h['Title'])
    HITId=h['HITId']
    response = mturk.list_assignments_for_hit(
        HITId=HITId,
        MaxResults=100,
        AssignmentStatuses=[
            'Submitted','Approved','Rejected'
        ]
    )
    try:
        print (json.dumps(response, indent=2))
    except:
        print(response)

3XJOUITW8URR3Y3M2XR5T2DDUSLTQR
This is a desktop Virtual Reality experiment
{'NextToken': 'p1:E+npAcGnF2L/UWnt6bgjlDh7Nuxt4m70NK4d09DTRcGj4idK0EkMm88N9PKyq3w=', 'NumResults': 1, 'Assignments': [{'AssignmentId': '3G2UL9A02DEDI2LO0NF76L0K4DD67N', 'WorkerId': 'AO18L4K2NAU0S', 'HITId': '3XJOUITW8URR3Y3M2XR5T2DDUSLTQR', 'AssignmentStatus': 'Rejected', 'AutoApprovalTime': datetime.datetime(2021, 4, 13, 12, 56, 46, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2021, 3, 14, 12, 56, 39, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2021, 3, 14, 12, 56, 46, tzinfo=tzlocal()), 'RejectionTime': datetime.datetime(2021, 3, 14, 12, 57, 35, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>response_code</QuestionIdentifier><FreeText>51c53943</FreeText></Answer></QuestionFormAnswers>', 'RequesterFeedback': "@@Sorry, coul

### Aprove all assignments in HIT

In [34]:
def approveAllAssignments(hit_id):
    # Get HIT status
    status=mturk.get_hit(HITId=hit_id)['HIT']['HITStatus']
    print('HITStatus:', status)
    if status == 'Reviewable':
        assignments = mturk.list_assignments_for_hit(HITId=hit_id, AssignmentStatuses=['Submitted'])
        if assignments['NumResults'] > 0:
            for assign in assignments['Assignments']:
                mturk.approve_assignment(AssignmentId=assign['AssignmentId'])
                print("Approved: "+assign['AssignmentId'])

### Aprove all assignments in all HITs

In [28]:
for item in mturk.list_hits()['HITs']:
    hit_id=item['HITId']
    approveAllAssignments(hit_id)

HITStatus: Assignable
HITStatus: Assignable
HITStatus: Assignable


# DANGER ZONE

### Force Delete HITS

In [ ]:
# Delete HIT
def deleteHit(hit_id):
    # Get HIT status
    status=mturk.get_hit(HITId=hit_id)['HIT']['HITStatus']
    print('HITStatus:', status)

    # If HIT is active then set it to expire immediately
    if status=='Assignable' or status=='Unassignable':
        print("Forcing it to expire")
        response = mturk.update_expiration_for_hit(
            HITId=hit_id,
            ExpireAt=datetime.datetime(2015, 1, 1)
        )        
    # Delete the HIT
    try:
        mturk.delete_hit(HITId=hit_id)
    except Exception as e: 
        print(e)
        print('Not deleted')
    else:
        print('Deleted')

### Delete All Hits

In [ ]:
for item in mturk.list_hits()['HITs']:
    hit_id=item['HITId']
    print('HITId:', hit_id)
    deleteHit(hit_id)

In [ ]:
for item in mturk.list_hits()['HITs']:
    print(f"HITId:{hit_id}")
    status=mturk.get_hit(HITId=item['HITId'])['HIT']['HITStatus']
    print(status)

# Loop and force delete all

In [ ]:
while len(mturk.list_hits()['HITs']):
    for item in mturk.list_hits()['HITs']:
        hit_id=item['HITId']
        print('HITId:', hit_id)
        deleteHit(hit_id)
    for item in mturk.list_hits()['HITs']:
        print(f"HITId:{hit_id}")
        status=mturk.get_hit(HITId=item['HITId'])['HIT']['HITStatus']
        print(status)
    print("Sleeping now")
    time.sleep(10)

# Deletes all the qualifications

In [ ]:
response = mturk.list_qualification_types(
    Query='participation',
    MustBeRequestable=False,
    MustBeOwnedByCaller=True,
    MaxResults=10
)
try:
    print (json.dumps(response, indent=2))
except:
    print(response)
qualification_list=response['QualificationTypes']
for qualification in qualification_list:
    qual_id=qualification['QualificationTypeId']
    print(qual_id)
    print(qualification['Name'])
    pp.pprint(qualification)
    response = mturk.delete_qualification_type(
        QualificationTypeId=qual_id
    )
    try:
        print (json.dumps(response, indent=2))
    except:
        print(response)